In [1]:
student_id = 'LCNet'

In [2]:
from competition_utils import *
from training_args import *

model_name = f'submission_{student_id}'
module_path = f"models.submission_20221555.{model_name}"
module = importlib.import_module(module_path)
globals()[model_name] = getattr(module, model_name)

Project_Name = 'Competition'
model_dir = 'models'
output_root = 'output'

SAVE_RESULT = True
SAVE_N = 1500
vis_root = f'vis/TEST_OUTPUTS_{datetime.now().strftime("%y%m%d_%H%M%S")}'

Dataset_root = 'Datasets'
Dataset_Name_list = [
                     #'VOC',
                     'ETIS',
                     #'CVPPP',
                     #'CFD',
                     #'CarDD',
]
number_of_classes_dict = {'VOC':21,
                          'ETIS':2,
                          'CVPPP':2,
                          'CFD':2,
                          'CarDD':2
                         }
epochs = 10
EARLY_STOP = 100
batch_size = 16
EXCLUDE_BACKGROUND = True
BINARY_SEG = None
THRESHOLD = 0.5
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
now = datetime.now()
Experiments_Time=now.strftime("%y%m%d_%H%M%S")
print('Experiment Start Time:',Experiments_Time)
eval_columns=['Experiment Time','Train Time', 'Dataset Name', 'Model Name', 'Val Loss', 'Test Loss', 'IoU', 'Dice',  'Precision', 'Recall', 'Total Params','Train-Predction Time']
df = pd.DataFrame(index=None, columns=eval_columns)
output_root = f'{output_root}/output_{Experiments_Time}'
os.makedirs(output_root, exist_ok = True)
seed = iteration = 1
for i in range(2):
    for j, Dataset_Name in enumerate(Dataset_Name_list):
        print(f'Dataset: {Dataset_Name} ({j+1}/{len(Dataset_Name_list)})')
        control_random_seed(seed)
        Dataset_dir = Dataset_root+'/'+Dataset_Name
        Height, Width = (256, 256)
        in_channels = 3
        number_of_classes = number_of_classes_dict[Dataset_Name]
        BINARY_SEG = True if number_of_classes==2 else False
        exclude_background = EXCLUDE_BACKGROUND
        out_channels = 1 if BINARY_SEG else number_of_classes

        train_image_path_list = natsort.natsorted(glob.glob(f"{Dataset_dir}/train/Originals/*"))
        train_target_path_list = natsort.natsorted(glob.glob(f"{Dataset_dir}/train/Masks/*"))
        validation_image_path_list = natsort.natsorted(glob.glob(f"{Dataset_dir}/val/Originals/*"))
        validation_target_path_list = natsort.natsorted(glob.glob(f"{Dataset_dir}/val/Masks/*"))
        test_image_path_list = natsort.natsorted(glob.glob(f"{Dataset_dir}/test/Originals/*"))
        test_target_path_list = natsort.natsorted(glob.glob(f"{Dataset_dir}/test/Masks/*"))
        print(f'train/val/test: {len(train_image_path_list)}/{len(validation_image_path_list)}/{len(test_image_path_list)}')

        # train_dataset = ImagesDataset(train_image_path_list, train_target_path_list, augment=False)
        train_dataset = ImagesDataset(train_image_path_list, train_target_path_list)
        validation_dataset = ImagesDataset(validation_image_path_list, validation_target_path_list)
        test_dataset = ImagesDataset(test_image_path_list, test_target_path_list)
        train_loader = SegDataLoader(
        train_dataset, batch_size=batch_size,
        num_workers=4, pin_memory=True, shuffle=True, drop_last=True, fill_last_batch=False,
        )
        validation_loader = SegDataLoader(
            validation_dataset, batch_size=batch_size, 
            num_workers=4, pin_memory=True,
        )
        test_loader = SegDataLoader(
            test_dataset, batch_size=batch_size, 
            num_workers=4, pin_memory=True,
        )

        print(f'{model_name}', end=' ')
        print(f'Dataset: {Dataset_Name}) ({j+1}/{len(Dataset_Name_list)})', end=' ')
        output_dir = output_root + f'/{model_name}_{Dataset_Name}'
        control_random_seed(seed)
        model=globals()[model_name](in_channels, out_channels)
        model = model.to(device)
        optimizer = Make_Optimizer(model)
        lr_scheduler = Make_LR_Scheduler(optimizer)
        criterion = Make_Loss_Function(number_of_classes)
        if SAVE_RESULT==True:
            os.makedirs(vis_root+f"/{Dataset_Name}", exist_ok=True)
        df = Execute_Experiment(model_name, model, Dataset_Name, train_loader, validation_loader, test_loader, 
                                optimizer, lr_scheduler, criterion, number_of_classes, df, epochs, device, output_dir,
                            BINARY_SEG, exclude_background, out_channels, seed, THRESHOLD, EARLY_STOP, SAVE_RESULT, vis_root+f"/{Dataset_Name}", Experiments_Time
                            )
        df.to_csv(output_root+'/'+f'{Project_Name}_'+Experiments_Time+'.csv', index=False, header=True, encoding="cp949")

Experiment Start Time: 250616_094747
Dataset: ETIS (1/1)
train/val/test: 118/39/39
submission_LCNet Dataset: ETIS) (1/1) Training Start Time: 250616_094748


Training:   0%|          | 0/7 [00:00<?, ?batch/s]

Valiation:   0%|          | 0/3 [00:00<?, ?batch/s]

1EP(250616_094749): T_Loss: 0.836702 V_Loss: 0.856145 IoU: 0.0796 Best Epoch: 1 Loss: 0.856145


Training:   0%|          | 0/7 [00:00<?, ?batch/s]

...

Training:   0%|          | 0/7 [00:00<?, ?batch/s]

Valiation:   0%|          | 0/3 [00:00<?, ?batch/s]

10EP(250616_094812): T_Loss: 0.637196 V_Loss: 0.680471 IoU: 0.2369 Best Epoch: 10 Loss: 0.680471
Test Start Time: 250616_094812


Test:   0%|          | 0/3 [00:00<?, ?batch/s]

Best Epoch: 10
Test(250616_094819): Loss: 0.640387 IoU: 0.2909 Dice: 0.3819 Precision: 0.3607 Recall: 0.5295
End 250616_094819
